In [ ]:
import pymc3 as pm
import pandas as pd
import matplotlib.pyplot as plt
from matplotlib.gridspec import GridSpec
import janitor as jn
from utils import ecdf
from sklearn.preprocessing import LabelEncoder

%load_ext autoreload
%autoreload 2
%matplotlib inline
%config InlineBackend.figure_format = 'retina'

## Problem Type

The Bayesian estimation model is widely applicable across a number of scenarios. The classical scenario is when we have an experimental design where there is a control vs. a treatment, and we want to know what the difference is between the two. Here, "estimation" is used to estimate the "true" value for the control and the "true" value for the treatment, and the "Bayesian" part refers to the computation of the uncertainty surrounding the parameter. 

Bayesian estimation's advantages over the classical t-test was first described by John Kruschke (2013). 

In this notebook, I provide a concise implementation suitable for two-sample and multi-sample inference.

## Data structure

To use it with this model, the data should be structured as such:

- Each row is one measurement.
- The columns should indicate, at the minimum:
    - What treatment group the sample belonged to.
    - The measured value.

## Extensions to the model

As of now, the model only samples posterior distributions of measured values. The model, then, may be extended to compute differences in means (sample vs. control) or effect sizes, complete with uncertainty around it. Use `pm.Deterministic(...)` to ensure that those statistics' posterior distributions, i.e. uncertainty, are also computed.

## Reporting summarized findings

Here are examples of how to summarize the findings.

> Treatment group A was greater than control by x units (95% HPD: [`lower`, `upper`]). 

> Treatment group A was higher than control (effect size 95% HPD: [`lower`, `upper`]). 

## Other notes

Here, we make a few modelling choices.

1. We care only about the `normalized_measurement` column, and so we choose the t-distribution to model it, as we don't have a good "mechanistic" model that incorporates measurement error of OD600 and 'measurement'.

In [ ]:
# Read in the data
df = pd.read_csv("../datasets/biofilm.csv").label_encode(
    columns=["isolate"]
)  # encode isolate as labels.

# Convert continuous columns to floatX for GPU compatibility.
continuous_cols = ["OD600", "ST", "replicate", "measurement", "normalized_measurement"]
for c in continuous_cols:
    df[c] = pm.floatX(df[c])

# Display a subset of the data.
df.head()

# Model Specification

Below, we define the model.

A visual representation of the model is below.

![](../images/biofilm.png)

In [ ]:
with pm.Model() as best:
    nu = pm.Exponential("nu_minus_one", lam=1 / 30) + 1

    fold = pm.Flat("fold", shape=len(set(df["isolate_enc"])))

    var = pm.HalfCauchy("var", beta=1, shape=len(set(df["isolate_enc"])))

    mu = fold[df["isolate_enc"].values]
    sd = var[df["isolate_enc"].values]

    like = pm.StudentT(
        "like", mu=mu, sd=sd, nu=nu, observed=df["normalized_measurement"]
    )

    # Compute differences
    diffs = pm.Deterministic("differences", fold[1:] - fold[0])

Sample from the posterior distribution.

In [ ]:
with best:
    trace = pm.sample(draws=2000, njobs=1)

Check for convergence using the traceplot.

In [ ]:
pm.traceplot(trace)

Looking at the traces, yes, everything looks more or less like a hairy caterpillar. This means that sampling went well, and has converged, thus we have a good MCMC estimator of the posterior distribution.

I need a mapping of isolate to its encoding - will come in handy below.

In [ ]:
mapping = dict(zip(df["isolate_enc"], df["isolate"]))
mapping

Let's use the Forest Plot to summarize how the strains differ from one another.

In [ ]:
ylabels = [mapping[i] for i in sorted(mapping.keys())]
pm.forestplot(trace, varnames=["fold"], ylabels=ylabels)

It's quite clear that:

- Strain 5 is very good at forming biofilms.
- Strain 1 (control strain) is not good at forming biofilms.

Let's look at the difference between the strains.

In [ ]:
pm.forestplot(trace, varnames=["differences"], ylabels=ylabels[1:])

Apart from Strain 2, none of the other strains' difference HPDs cross "zero".

This means that they all would be considered to be "statistically significantly different" from strain 1.

Let's check the variances, just to see if there's a difference in that.

In [ ]:
pm.forestplot(trace, varnames=["var"], ylabels=ylabels)

It is clear that there are a number of strains for whom the variance is similar. Without better information, though, we would not be warranted to impose a similarity structure on the data.

# Model Check

Let's see if we can generate PPC samples that look similar. Admittedly, there's a bit of an art to checking here - there's only 6 measurements per strain, so it's not like we have a lot of data to work with.

In [ ]:
samples = pm.sample_ppc(trace, model=best)
samples["like"].shape

In [ ]:
# We want indices for each of the samples.
indices = dict()
for enc, iso in mapping.items():
    idxs = list(df[df["isolate_enc"] == enc].index)
    indices[iso] = idxs
indices

In [ ]:
samples["like"][:, idxs].mean(axis=1)

In [ ]:
# Make PPC plot for one of the groups.
fig = plt.figure(figsize=(16, 16))
gs = GridSpec(nrows=4, ncols=4)
axes = dict()


for i, (strain, idxs) in enumerate(indices.items()):
    if i > 0:
        ax = fig.add_subplot(gs[i], sharex=axes[0])
    else:
        ax = fig.add_subplot(gs[i])
    x, y = ecdf(df.iloc[idxs]["normalized_measurement"])
    ax.plot(x, y, label="data")
    x, y = ecdf(samples["like"][:, idxs].mean(axis=1))
    ax.plot(x, y, label="ppc")
    ax.set_title(f"Strain {strain}")
    axes[i] = ax

While the PPC samples are generally okay, the StudentT distribution does give some long-tail values, including those that are negative. Given the measurements at hand, negative values would be considered "absurd" values. 

If I had more time, I might experiment with the use of a different likelihood distribution. However, this is a good enough first start model.